# 1. Establish communication with the card

## 1.1. Find the smartcard reader

In [1]:
from applets.core import get_connection
from applets.memorycard import MemoryCard

In [2]:
connection = get_connection()
if connection is None:
    print("Failed to open a connection. No cardreaders?")

## 1.2. Power on the card and get `ATR`
`ATR` - Answer To Reset, bytes retured by the card when it is powered on.<br>
It helps to determine communication protocol and frequency we should choose.

In [3]:
atr=bytes(connection.getATR())
print(atr.hex())

3bdc18ff8191fe1fc38073c821136605036351000250


## 1.3. Select the applet by AID
JavaCard can have multiple applets installed. They are like very isolated "apps".<br>
They can't talk to each other and only one can be active at any time.

In [4]:
app = MemoryCard(connection)
app.select()

b''

In [5]:
# get random data without secure communication
app.get_random().hex()

'19de4d8abd0bedd89ff22798df36f00d63fc9f00df60113ea8cf4bbc49ece680'

## 1.4. Establish secure communication

In [10]:
app.establish_secure_channel(mode="se") # mode="se" and mode="ee" available, "se" by default

In [11]:
app.is_secure_channel_open

True

# 2. Start talking to the card securely

In [12]:
# sends encrypted data to the card, responce has the same data
app.echo(b'ping')

b'ping'

In [13]:
# get 32 bytes of randomness over secure channel
app.get_random_sc()

b'\xe5\x01AbO-\xd2\xdf\xc3L\xc2\x1c\x02\x8a\xf6M\x19\x98\xd4L\xa4n\xd3z\xe7\xd0\xa4E\x94l\xb4\xe5'

In [ ]:
# get status
app.secure_request(b'\x03\x00').hex()

In [ ]:
# unlock with pin, or set pin. Pin can be anything up to 32 bytes. Want more? - hash it.
app.secure_request(b'\x03\x01'+b'q')

In [ ]:
# if you are setting new pin - call it twice - first to set pin, second - to unlock card.
app.secure_request(b'\x03\x01'+b'q')

In [ ]:
# change pin
app.secure_request(b'\x03\x03'+b'\x01'+b'q'+b'\x06'+b'passwd')

In [ ]:
# unlock with new pin
app.secure_request(b'\x03\x01'+b'passwd')

In [ ]:
# get data over secure channel
app.get_secret_data()

In [ ]:
# set data over secure channel
mnemonic = b'universe universe universe universe universe universe universe universe universe universe universe universe universe universe universe universe universe universe universe universe universe universe universe urge'
app.put_secret_data(mnemonic)

In [ ]:
# check the new data is stored now
app.get_secret_data()

In [ ]:
# lock card
app.secure_request(b'\x03\x02')

In [14]:
app.close_secure_channel()

In [15]:
connection.disconnect()